In [14]:
import xml.etree.ElementTree as ET
from collections import defaultdict
import pprint
import re

In [15]:
filename = 'example.xml'
tag_types = {}
for event, elem in ET.iterparse(filename):
    tag = elem.tag
    if tag not in tag_types.keys():
        tag_types[tag] = 1
    else:
        tag_types[tag] += 1
        
print(tag_types)

{'bounds': 1, 'node': 20, 'tag': 7, 'nd': 4, 'way': 1, 'member': 3, 'relation': 1, 'osm': 1}


In [16]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        value = element.attrib['k']
        print(value)
        if problemchars.search(value):
            keys['problemchars'] += 1
        elif lower.search(value):
            keys['lower'] += 1
        elif lower_colon.search(value):
            keys['lower_colon'] += 1
        else:
            keys['other'] += 1
         
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

process_map(filename)

highway
amenity
cuisine
name
highway
restriction
type


{'lower': 7, 'lower_colon': 0, 'other': 0, 'problemchars': 0}

In [17]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

In [30]:
filename = 'exampleStreetName.xml'

In [31]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
           "Rd." : "Road",
           "Rd" : "Road",
           "Ave": "Avenue",
           "Ave." :"Avenue"
            }

In [32]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):

    name = name.split(" ")
    if name[-1] in mapping.keys():
        name = mapping[name[-1]]

    return name

street_types = audit(filename)

In [33]:
street_types

defaultdict(set,
            {'Ave': {'N. Lincoln Ave', 'North Lincoln Ave'},
             'Rd.': {'Baldwin Rd.'},
             'St.': {'West Lexington St.'}})

In [45]:
for _, elem in ET.iterparse(filename):
    if elem.tag=='node':
        for tag in elem.iter('tag'):
            if len(tag.attrib) == 0:
                print(None)
            else: 
                print(tag.attrib)

{'k': 'highway', 'v': 'traffic_signals'}
{'k': 'addr:city', 'v': 'Chicago'}
{'k': 'addr:housenumber', 'v': '5157'}
{'k': 'addr:postcode', 'v': '60625'}
{'k': 'addr:street', 'v': 'North Lincoln Ave'}
{'k': 'amenity', 'v': 'restaurant'}
{'k': 'cuisine', 'v': 'mexican'}
{'k': 'name', 'v': 'La Cabana De Don Luis'}
{'k': 'outdoor_seating', 'v': 'no'}
{'k': 'phone', 'v': '1 (773)-271-5176'}
{'k': 'smoking', 'v': 'no'}
{'k': 'takeaway', 'v': 'yes'}
{'k': 'addr:city', 'v': 'Chicago'}
{'k': 'addr:country', 'v': 'US'}
{'k': 'addr:housenumber', 'v': '4874'}
{'k': 'addr:postcode', 'v': '60625'}
{'k': 'addr:state', 'v': 'Illinois'}
{'k': 'addr:street', 'v': 'N. Lincoln Ave'}
{'k': 'name', 'v': 'Matty Ks'}
{'k': 'phone', 'v': '(773)-654-1347'}
{'k': 'shop', 'v': 'doityourself'}
{'k': 'source', 'v': 'GPS'}
{'k': 'amenity', 'v': 'fast_food'}
{'k': 'cuisine', 'v': 'sausage'}
{'k': 'name', 'v': "Shelly's Tasty Freeze"}
{'k': 'addr:housename', 'v': 'Village Hall'}
{'k': 'addr:housenumber', 'v': '1400'}
{

In [40]:
node_id

'1683602133'